# Lab 9 - DEFI(Uniswap V2)

## 1. Start Hardhat Node

### Step 1. Open a terminal

### Step 2. Run Hardhat Node in lab-9

```sh
hh node
```

## 2. Deploy Contracts Locally

In [ ]:
const { ethers } = require("hardhat");

new Promise(async (res, rej) => {
    
    account = await ethers.getSigner();

    // Deploy uniswap contracts
    const Factory = await ethers.getContractFactory("UniswapV2Factory");
    factory = await Factory.deploy(account.address);

    const Weth = await ethers.getContractFactory("WETH9");
    weth = await Weth.deploy();

    const UniswapV2Router = await ethers.getContractFactory(
        "UniswapV2Router02"
    );
    router = await UniswapV2Router.deploy(factory.address, weth.address);

    // Deploy TokenA and TokenB
    const ERC20 = await ethers.getContractFactory(
        "./contracts/v2-periphery/test/ERC20.sol:ERC20"
    );
    tokenA = await ERC20.deploy(ethers.utils.parseEther("2"));
    tokenB = await ERC20.deploy(ethers.utils.parseEther("2"));

    contracts = {
        WETH: weth.address,
        UNISWAPV2_FACTORY: factory.address,
        UNISWAPV2_ROUTER: router.address,
        TOKEN_A: tokenA.address,
        TOKEN_B: tokenB.address,
    };
    console.log(contracts);
});


## 3. Load Contracts

In [ ]:
new Promise(async (res,rej)=>{

    account = await ethers.getSigner();

    tokenA = await ethers.getContractAt(
        "./contracts/v2-periphery/test/ERC20.sol:ERC20",
        contracts.TOKEN_A
    );
    
    tokenB = await ethers.getContractAt(
        "./contracts/v2-periphery/test/ERC20.sol:ERC20",
        contracts.TOKEN_B
    );

    factory = await ethers.getContractAt(
        "UniswapV2Factory",
        contracts.UNISWAPV2_FACTORY
    );


    router = await ethers.getContractAt(
        "UniswapV2Router02",
        contracts.UNISWAPV2_ROUTER
    );    

})


## 3. Add Liquidity

We can leave the minimum amount as zero since the pool does not exist.
Set the timestamp deadline to 1 minute from current block timestamp since we are only running on the Development Network.

In [ ]:
new Promise(async(res,rej)=>{
    await tokenA.approve(contracts.UNISWAPV2_ROUTER, 10_000);
    await tokenB.approve(contracts.UNISWAPV2_ROUTER, 10_000);
    
    const ts = (await ethers.provider.getBlock()).timestamp + 1000;
    const response = await router.addLiquidity(
        contracts.TOKEN_A,
        contracts.TOKEN_B,
        10_000,
        10_000,
        0,
        0,
        account.address,
        ts
    );

})



## 4. Check Pool

The Uniswap Router will call the Uniswap Factory to create the pool and call the PairCreated event. We can check the factory's event log to get the pool address. This address can be used to create a Pool contract instance to query the reserves.

In [ ]:
new Promise(async (res,rej)=>{
    const logs = await factory.queryFilter(
        factory.filters.PairCreated(null)
    );
    poolAddr = logs[0].args.pair;
    console.log(poolAddr);
})


## 5. Create the getReserves() Function

Knowing the reserve balance in the pool is a key function in using Uniswap. It is used to confirm that the reserve is indeed in the pool after funding. It is also used to derive the prices for the next lab in making trade decisions. Getting the reserve balance is a 2 step process: find the pool address from the factory, and get the reserve from the pool.

In [ ]:
const getReserves = async (tokenA, tokenB) => {
   const poolAddr = await factory.getPair(tokenA.address, tokenB.address);
   const pool = await ethers.getContractAt("UniswapV2Pair", poolAddr);
   const { _reserve0, _reserve1 } = await pool.getReserves();
       return tokenA.address < tokenB.address
        ? { reserveA: _reserve0, reserveB: _reserve1 }
        : { reserveA: _reserve1, reserveB: _reserve0 };
}

In [ ]:
new Promise(async (res,rej)=>{
    const reserves = await getReserves(tokenA, tokenB);
    console.log(reserves);
})


## 6. Create Token Swap

In the previous lab, we have created a reserve of 10000 tokenA and tokenB by the liquidity provider. In this lab, we will simulate a trader using accounts[1] to trade 2000 units of TokenA for TokenB. We will use the reserve balanaces to compute the amount of TokenB we can receive from this trade, ie. its execution price.


### Step 1. Create getAmountOut() function

This function is used to compute the expected amount of token output (TokenB) when given an exact amount of token input (TokenA).

**a)  Refer to the `GetAmountOut()` Solidity function in `v2-periphery/UniswapV2Library.sol`**

This corresponds to the sell formula for computing $\Delta{y}$ as explained in Lesson 9.

<center>

$\Delta{y} = \frac{yr\Delta{x}}{x + r\Delta{x}}$

</center>


**b)  Convert the logic to Javascript**

In [ ]:
const { BigNumber } = require("ethers");
const getAmountOut = (amountIn, reserveIn, reserveOut) => {
    amountIn = BigNumber.from(amountIn);
    const amountInWithFee = amountIn.mul(997);
    const numerator = amountInWithFee.mul(reserveOut);
    const denominator = reserveIn.mul(1000).add(amountInWithFee);
    amountOut = numerator / denominator;
    return BigNumber.from(Math.floor(amountOut).toString());
};


### Step 2. Create Mock Trader

Create a mock trader using accounts[1] and transfer some tokenA and tokenB to its account.

In [ ]:
new Promise(async(res,rej)=>{
    trader = await ethers.getSigner(1);
    await tokenA.transfer(trader.address, ethers.utils.parseEther("1"));
    await tokenB.transfer(trader.address, ethers.utils.parseEther("1"));
})

### Step 3. Check Trader Balance Before Swap

In [ ]:
new Promise(async(res,rej)=>{
    console.log(`Trader: tokenA before=`, await tokenA.balanceOf(trader.address));
    console.log(`Trader: tokenB before=`, await tokenB.balanceOf(trader.address));
})


### Step 4. Calculate the Swap Amount

We want to pay 2000 tokenA for tokenB.

We can use the getAmountOut function to find the amount of TokenB to receive. However, in actual trading scenario the price will fluctuate due to slippage. So we will call the getAmountOut function to compute the minimum tokenB amount expected and set 95% of this amount as the tolerance. If the slippage cause the TokenB received to be less than 95% then the trade will be rejected.

In [ ]:
new Promise(async(res,rej)=>{
    reservesBefore = await getReserves(tokenA, tokenB);
    console.log("Pool: reserves before=", reservesBefore);
    
    amtA = 2000;
    amtB = getAmountOut(amtA, reservesBefore.reserveA, reservesBefore.reserveB);
    minAmtB = Math.floor(0.95 * amtB);

    console.log(`Amount In: ${amtA} TokenA`);
    console.log(`Amount Out: ${amtB} TokenB`);
    console.log(`Min. Amount Out: ${minAmtB} TokenB`);
})



### Step 5. Execute the trade

The trader must approve Uniswap Router to withdraw 2000 TokenA from his account for the trade before executing the trade.

We will execute the trade using the Uniswap Router contracts **swapExactTokensForTokens()** function.

The `path` parameter of the function expects an array. The first element of the array is the token address to send (TokenA) and the second element of the array is the token address to receive (TokenB).

In [ ]:
new Promise(async(res,rej)=>{
    // Approve router to withdraw 2000 TokenA from trader account
    await tokenA.connect(trader).approve(router.address, amtA);

    // Trade 2000 TokenA for 1662 TokenB using trader account.
    // Cancel the trade if TokenB amount is less than 1578
    const ts = (await ethers.provider.getBlock()).timestamp + 1000;
    await router
        .connect(trader)
        .swapExactTokensForTokens(
            amtA,
            minAmtB,
            [tokenA.address, tokenB.address],
            trader.address,
            ts
        );
})

### Step 6. Check Balance After Swap

In [ ]:
new Promise(async(res,rej)=>{
    // Check Balance After
    console.log(`Trader: tokenA after=`, await tokenA.balanceOf(trader.address));
    console.log(`Trader: tokenB after=`, await tokenB.balanceOf(trader.address));
    
    // Check reserves after
    const reservesAfter = await getReserves(tokenA, tokenB);
    console.log("Pool: reserves after=", reservesAfter);

})



The output should show:

   ```sh
   Trader: tokenA before=1000000000000000000
   Trader: tokenB before=1000000000000000000
   Pool: reserves before = { reserveA: 10000, reserveB: 10000}
   Trader: tokenA after=999999999999998000
   Trader: tokenB after=1000000000000001662
   Pool: reserves after = { reserveA: 12000, reserveB: 8338}
   ```

- The trader starts off with 1 ETH of TokenA and TokenB and the resulting balance after paying 2000 TokenA and receiving 1662 TokenB.

- The Pool's reserves shows that it has increased TokenA reserve by 2000 and reduced TokenB's reserved by 1662.


## 7. Remove Liquidity

In this lab, we will analyse the effect of cashing out the Liquidity Tokens.

### Step 1. Check Liquidity Token Balance

Get the current Liquidity Token balance for the Liquidity Provider.

In [ ]:
new Promise(async(res,rej)=>{
    const poolAddr = await factory.getPair(tokenA.address, tokenB.address);
    const pool = await ethers.getContractAt("UniswapV2Pair", poolAddr);

    // Check LP's liquidity token balance
    const lpBefore = await pool.balanceOf(account.address);
    console.log("LP: liquidity before=", lpBefore);
    const lpTokenABefore = await tokenA.balanceOf(account.address);
    console.log("LP: tokenA before=", lpTokenABefore);
    const lpTokenBBefore = await tokenB.balanceOf(account.address);
    console.log("LP: tokenB before=", lpTokenBBefore);
    const reservesBefore = await getReserves(tokenA, tokenB);
    console.log("Pool: reserves before=", reservesBefore);

})

### Step 2. Cash out

In [ ]:
new Promise(async (res,rej)=>{
    const poolAddr = await factory.getPair(tokenA.address, tokenB.address);
    const pool = await ethers.getContractAt("UniswapV2Pair", poolAddr);
    const lpBefore = await pool.balanceOf(account.address);
    
    // Approve router to withdraw all LP tokens
    await pool.approve(router.address, lpBefore);
    
    // Remove Liquidity. Liquidate all LP's liquidity tokens
    const ts = (await ethers.provider.getBlock()).timestamp + 1000;

    await router.removeLiquidity(
        tokenA.address,
        tokenB.address,
        lpBefore,
        0,
        0,
        account.address,
        ts
    );

})


### Step 3. Check Liquidity Token Balance Final

In [ ]:
new Promise(async (res,rej)=>{

    const poolAddr = await factory.getPair(tokenA.address, tokenB.address);
    const pool = await ethers.getContractAt("UniswapV2Pair", poolAddr);


    console.log("AFTER--------");
    const lpAfter = await pool.balanceOf(account.address);
    console.log("LP: liquidity After=", lpAfter);
    const lpTokenAAfter = await tokenA.balanceOf(account.address);
    console.log("LP: tokenA After=", lpTokenAAfter);
    const lpTokenBAfter = await tokenB.balanceOf(account.address);
    console.log("LP: tokenB After=", lpTokenBAfter);
    const reservesAfter = await getReserves(tokenA, tokenB);
    console.log("Pool: reserves After=", reservesAfter);

})



The output shows:

```sh
LP: tokenA Balance=1000000000000000800
LP: tokenB Balance=999999999999997504
LP: Liquidity Balance=0
Pool: reserveA = 1200
Pool: reserveB = 834
Pool: midPrice of TokenA = 0.695 TokenB
Pool: midPrice of TokenB = 1.4388489208633093 TokenA
```

* Before liquidation, the pool has reserveA = 12000 and reserveB = 8338

*  After liquidation, the pool is left with = 1200 and reserveB = 834

This is because the pool was originally funded with 10_000 but 1000 is permanently locked. That means the liquidity provider's share of the pool is only 90%. Liquidating 9000 liquidity tokens means the pool is left with 10% of both tokens.

## OPTIONAL - Trace the transactions

So far, we know how to use the contract to create a liquidity pool and receive the Liquidity Token. But we still don't know how the contract works. To look inside the contract, we can run a trace.

1. Install hardhat-tracer plugin

```js
npm i -D hardhat-tracer@1.1.0-rc.6
```

2. Run the trace

```js
hh test test/test1-add-liquidity --trace
```

![add-liquidity-trace](img/add-liquidity-trace.png)

3. Analyse the trace under the **UniswapV2Router02.addLiquidity()** section

-   **EVENT UniswapV2Factory.PairCreated** shows that a pool has been created where token0 = TokenB address, token1 = TokenA address.
-   The 2 **EVENT ERC20.Transfer** events after that shows that 10,000 tokens has been transferred into the pool `0x82404cCfc1c17Ee7b6a8bEde74da9BBca9cE96Af`.
-   The next 2 **EVENT ERC20.Transfer** events shows that 1,000 Liquidity Tokens are minted and sent to address(0) while the remaining 9,000 tokens are minted into the liquidity provider's address.
-   The **EVENT UniswapV2Pair.Sync** event shows that the reserve balance has been updated to match the transfer amount.
-   Finally, the **EVENT UniswapV2Pair.Mint** event reports the result of minted by UniswapV2Router02 (sender=0x5797b2170589F01573064F9683c035554d47ffd3)

4. Analyse the trace under **UniswapV2Router02.swapExactTokensForTokens()**

![uniswap-trade-trace](img/uniswap-trade-trace.png)

-   The first **ERC20.Transfer()** event is a withdrawal of 2000 TokenA from trader's account into the pool.
-   The router than call the **UniswapV2Pair.swap()** function which transfers 1662 TokenB from the pool's account to the trader.
-   Finally, it updates the pool's reserve with TokenB=8338 and TokenA=12000.
